In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
n_gpus = 1
model_name = 'inceptionresnetv2_fine_tuning_8clf_2'

import keras.backend as K
import tensorflow as tf

# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3)
# sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

Using TensorFlow backend.


# 导入必要的库

In [2]:
import numpy as np
import pandas as pd
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import l2

from keras.preprocessing.image import *

import matplotlib.pyplot as plt
import random
import os
import cv2
from tqdm import tqdm
from glob import glob
import multiprocessing

from collections import Counter
from keras import backend as K
from keras.utils import multi_gpu_model

from IPython.display import display, Image

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# 读取数据集

In [3]:
df = pd.read_csv('../../data/base/Annotations/label_new.csv', header=None)
df.columns = ['filename', 'label_name', 'label']
df = df.sample(frac=1).reset_index(drop=True) # shuffle

df.label_name = df.label_name.str.replace('_labels', '')

display(df.head())
print(len(df))
c = Counter(df.label_name)
c

,filename,label_name,label
0,Images/neck_design_labels/1520a2f84b24fc80c04f...,neck_design,ynnnn
1,Images/pant_length_labels/b8de2ff7afd43523fe61...,pant_length,nnnnny
2,Images/sleeve_length_labels/085805ad2138b725d2...,sleeve_length,nnnnymnnn
3,Images/skirt_length_labels/d8e9b4d496a5ae6f1f6...,skirt_length,nnnnyn
4,Images/neckline_design_labels/7de682287502f8d2...,neckline_design,nnnnnnynnn


89683


Counter({'coat_length': 11320,
         'collar_design': 8393,
         'lapel_design': 7034,
         'neck_design': 5696,
         'neckline_design': 17148,
         'pant_length': 7460,
         'skirt_length': 19333,
         'sleeve_length': 13299})

In [4]:
label_count = dict([(x, len(df[df.label_name == x].label.values[0])) for x in c.keys()])
label_names = list(label_count.keys())
display(label_count)

{'coat_length': 8,
 'collar_design': 5,
 'lapel_design': 5,
 'neck_design': 5,
 'neckline_design': 10,
 'pant_length': 6,
 'skirt_length': 6,
 'sleeve_length': 9}

## 生成 y

In [5]:
fnames = df['filename'].values
width = 331
n = len(df)
y = [np.zeros((n, label_count[x])) for x in label_count.keys()]
for i in range(n):
    label_name = df.label_name[i]
    label = df.label[i]
    y[label_names.index(label_name)][i, label.find('y')] = 1
    

## 读取图片

In [6]:
def f(index):
    return index, cv2.resize(cv2.imread('../../data/base/'+fnames[index]), (width, width))

X = np.zeros((n, width, width, 3), dtype=np.uint8)
with multiprocessing.Pool(32) as pool:
    with tqdm(pool.imap_unordered(f, range(n)), total=n) as pbar:
        for i, img in pbar:
            X[i] = img[:,:,::-1]

100%|██████████| 89683/89683 [01:06<00:00, 1353.80it/s]


In [7]:
n_train = int(n*0.9)
X_train = X[:n_train]
X_valid = X[n_train:]
y_train = [x[:n_train] for x in y]
y_valid = [x[n_train:] for x in y]

In [8]:
def display_images(imgs, w=8, h=4, figsize=(24, 12)):
    plt.figure(figsize=figsize)
    for i in range(w*h):
        plt.subplot(h, w, i+1)
        plt.imshow(imgs[i])

In [9]:
class Generator():
    def __init__(self, X, y, batch_size=8, aug=False):
        def generator():
            idg = ImageDataGenerator(horizontal_flip=True,
                                     rotation_range=20,
                                     zoom_range=0.2)
            while True:
                for i in range(0, len(X), batch_size):
                    X_batch = X[i:i+batch_size].copy()
                    y_barch = [x[i:i+batch_size] for x in y]
                    if aug:
                        for j in range(len(X_batch)):
                            X_batch[j] = idg.random_transform(X_batch[j])
                    yield X_batch, y_barch
        self.generator = generator()
        self.steps = len(X) // batch_size + 1

In [10]:
gen_train = Generator(X_train, y_train, batch_size=8, aug=True)

# 数据集探索

## 类别分布

In [11]:
# plt.figure(figsize=(26, 14))
# for i in range(8):
#     plt.subplot(2, 4, i+1)
#     counts = Counter(y[i].argmax(axis=-1)[np.where(y[i].any(axis=-1))])
#     pd.Series(counts).plot('bar')

### 抽样可视化

In [12]:
# plt.figure(figsize=(26, 14))
# w = 8
# h = 4
# for i in range(w*h):
#     plt.subplot(h, w, i+1)
#     index = np.random.randint(n)
#     plt.title(str([y[x][index].argmax() if y[x][index].any() else -1 for x in range(8) ]))
#     plt.imshow(X[index])
#     plt.axis('off')

# 搭建模型并训练

## 搭建模型

In [ ]:
def acc(y_true, y_pred):
    index = tf.reduce_any(y_true > 0.5, axis=-1)
    res = tf.equal(tf.argmax(y_true, axis=-1), tf.argmax(y_pred, axis=-1))
    index = tf.cast(index, tf.float32)
    res = tf.cast(res, tf.float32)
    return tf.reduce_sum(res * index) / (tf.reduce_sum(index) + 1e-7)

In [ ]:
base_model = InceptionResNetV2(weights='imagenet', input_shape=(width, width, 3), include_top=False, pooling='avg')

input_tensor = Input((width, width, 3))
x = input_tensor
x = Lambda(inception_resnet_v2.preprocess_input)(x)
x = base_model(x)
x = Dropout(0.5)(x)
x = [Dense(count, activation='softmax', name=name)(x) for name, count in label_count.items()]

model = Model(input_tensor, x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot, plot_model

# plot_model(model, show_shapes=True, to_file='model_simple.png')
# SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
# model2 = multi_gpu_model(model, n_gpus)

In [ ]:
# opt = SGD(1e-3, momentum=0.9, nesterov=True, decay=1e-5)

In [ ]:
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=[acc])
model.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=2, validation_data=(X_valid, y_valid))

In [18]:
model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=[acc])
model.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=3, validation_data=(X_valid, y_valid))

Epoch 1/3
10090/10090 [==============================] - 7094s 703ms/step - loss: 0.3496 - neck_design_loss: 0.0300 - lapel_design_loss: 0.0233 - neckline_design_loss: 0.0796 - collar_design_loss: 0.0332 - sleeve_length_loss: 0.0493 - skirt_length_loss: 0.0576 - pant_length_loss: 0.0254 - coat_length_loss: 0.0511 - neck_design_acc: 0.3372 - lapel_design_acc: 0.4369 - neckline_design_acc: 0.7088 - collar_design_acc: 0.4747 - sleeve_length_acc: 0.6398 - skirt_length_acc: 0.7813 - pant_length_acc: 0.4490 - coat_length_acc: 0.5633 - val_loss: 0.3752 - val_neck_design_loss: 0.0357 - val_lapel_design_loss: 0.0282 - val_neckline_design_loss: 0.0904 - val_collar_design_loss: 0.0423 - val_sleeve_length_loss: 0.0434 - val_skirt_length_loss: 0.0527 - val_pant_length_loss: 0.0287 - val_coat_length_loss: 0.0538 - val_neck_design_acc: 0.3451 - val_lapel_design_acc: 0.4087 - val_neckline_design_acc: 0.6776 - val_collar_design_acc: 0.4893 - val_sleeve_length_acc: 0.6309 - val_skirt_length_acc: 0.7795 

In [19]:
model.compile(optimizer=Adam(1e-6), loss='categorical_crossentropy', metrics=[acc])
model.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=1, validation_data=(X_valid, y_valid))

Epoch 1/1
10090/10090 [==============================] - 7106s 704ms/step - loss: 0.2137 - neck_design_loss: 0.0192 - lapel_design_loss: 0.0123 - neckline_design_loss: 0.0502 - collar_design_loss: 0.0210 - sleeve_length_loss: 0.0292 - skirt_length_loss: 0.0339 - pant_length_loss: 0.0160 - coat_length_loss: 0.0320 - neck_design_acc: 0.3608 - lapel_design_acc: 0.4600 - neckline_design_acc: 0.7517 - collar_design_acc: 0.5034 - sleeve_length_acc: 0.6730 - skirt_length_acc: 0.8123 - pant_length_acc: 0.4719 - coat_length_acc: 0.6023 - val_loss: 0.3537 - val_neck_design_loss: 0.0346 - val_lapel_design_loss: 0.0259 - val_neckline_design_loss: 0.0867 - val_collar_design_loss: 0.0416 - val_sleeve_length_loss: 0.0408 - val_skirt_length_loss: 0.0469 - val_pant_length_loss: 0.0266 - val_coat_length_loss: 0.0506 - val_neck_design_acc: 0.3523 - val_lapel_design_acc: 0.4185 - val_neckline_design_acc: 0.6942 - val_collar_design_acc: 0.5031 - val_sleeve_length_acc: 0.6422 - val_skirt_length_acc: 0.7886 

# 计算验证集准确率

In [20]:
model.save_weights('../../data/model_incres_weights.h5')

In [21]:
y_pred = model.predict(X_valid, batch_size=64, verbose=1)
a = np.array([x.any(axis=-1) for x in y_valid]).T.astype('uint8')
b = [np.where((a == np.eye(8)[x]).all(axis=-1))[0] for x in range(8)]
for c in range(8):
    y_pred2 = y_pred[c][b[c]].argmax(axis=-1)
    y_true2 = y_valid[c][b[c]].argmax(axis=-1)
    print(label_names[c], (y_pred2 == y_true2).mean())

8969/8969 [==============================] - 217s 24ms/step
neck_design 0.835294117647
lapel_design 0.90261627907
neckline_design 0.870206489676
collar_design 0.87443438914
sleeve_length 0.905426356589
skirt_length 0.927056389033
pant_length 0.889495225102
coat_length 0.87054735013


In [22]:
counts = Counter(df.label_name)
counts

Counter({'coat_length': 11320,
         'collar_design': 8393,
         'lapel_design': 7034,
         'neck_design': 5696,
         'neckline_design': 17148,
         'pant_length': 7460,
         'skirt_length': 19333,
         'sleeve_length': 13299})

In [23]:
s = 0
n = 0
for c in range(8):
    y_pred2 = y_pred[c][b[c]].argmax(axis=-1)
    y_true2 = y_valid[c][b[c]].argmax(axis=-1)
    s += counts[label_names[c]] * (y_pred2 == y_true2).mean()
    n += counts[label_names[c]]
print(s / n)

0.890052112243


In [ ]:
df_test = pd.read_csv('../../data/z_rank/Tests/question.csv', header=None)
df_test.columns = ['filename', 'label_name', 'label']

fnames_test = df_test.filename

n_test = len(df_test)
df_test.head()

In [ ]:
def f(index):
    return index, cv2.resize(cv2.imread('../../data/z_rank/'+fnames_test[index]), (width, width))

X_test = np.zeros((n_test, width, width, 3), dtype=np.uint8)
with multiprocessing.Pool(12) as pool:
    with tqdm(pool.imap_unordered(f, range(n_test)), total=n_test) as pbar:
        for i, img in pbar:
            X_test[i] = img[:,:,::-1]

In [ ]:
y_pred = model.predict(X_test, batch_size=64, verbose=1)

In [ ]:
for i in range(n_test):
    problem_name = df_test.label_name[i].replace('_labels', '')
    problem_index = label_names.index(problem_name)
    probs = y_pred[problem_index][i]
    df_test.label[i] = ';'.join(np.char.mod('%.8f', probs))

In [ ]:
fname_csv = '../../data/pred_incres.csv'
fname_csv_model = '../../data/pred_incres_model.csv'
df_test.to_csv(fname_csv, index=None, header=None)
df_test.to_csv(fname_csv, index=None, header=None)